In [143]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import sys
sys.path.append('../')
import src.cleaning_functions as cf
import pyjsonviewer
from functools import reduce
import operator
from pymongo import MongoClient
from pymongo import GEOSPHERE
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

In [2]:
load_dotenv()

True

#### Utilizo los token de la API de Foursquare para poder llamarla 

In [3]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
***30% of the company staff have at least 1 child.**
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
***Account managers need to travel a lot.**
***Everyone in the company is between 25 and 40, give them some place to go party.**
***The CEO is vegan.**
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

## MADRID

#### Utilizo una función para buscar las coordenadas del punto concreto elegido para la ciudad de Madrid

In [5]:
#todo= {"nombre": ["name"], "dirección": ["location", "address"], "latitud": ["location", "lat"], "longitud": ["location", "lng"]}

In [6]:
cf.geocode("Castellana 81, Paseo de la Castellana, Madrid")

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

In [7]:
Madrid = cf.geocode("Castellana 81, Paseo de la Castellana, Madrid")

In [8]:
Madrid

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

### Airports

In [9]:
parametros_airp = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [10]:
airports = requests.get(url_query, params = parametros_airp).json()

In [11]:
airports

{'meta': {'code': 200, 'requestId': '6190ecbffdc9c60ce9dd1f01'},
 'response': {'venues': [{'id': '4afde737f964a520e72b22e3',
    'name': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
    'contact': {},
    'location': {'address': 'Av. de la Hispanidad, s/n',
     'lat': 40.4902221628413,
     'lng': -3.569434148638026,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.4902221628413,
       'lng': -3.569434148638026}],
     'distance': 11590,
     'postalCode': '28042',
     'cc': 'ES',
     'city': 'Madrid',
     'state': 'Madrid',
     'country': 'España',
     'formattedAddress': ['Av. de la Hispanidad, s/n',
      '28042 Madrid Madrid',
      'España']},
    'categories': [{'id': '4bf58dd8d48988d1ed931735',
      'name': 'Airport',
      'pluralName': 'Airports',
      'shortName': 'Airport',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
       'suffix': '.png'},
      'primary': True}],
  

In [12]:
airports.keys()

dict_keys(['meta', 'response'])

In [13]:
airports["response"].keys()

dict_keys(['venues'])

In [14]:
airports["response"]["venues"][0]["name"]

'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)'

In [15]:
data = airports["response"]["venues"]

In [16]:
df = pd.DataFrame(data)
df

,id,name,contact,location,categories,verified,stats,beenHere,hereNow,referralId,venueChains,hasPerk,venuePage
0,4afde737f964a520e72b22e3,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,{},"{'address': 'Av. de la Hispanidad, s/n', 'lat'...","[{'id': '4bf58dd8d48988d1ed931735', 'name': 'A...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 4, 'summary': '4 people are here', '...",v-1636887743,[],False,NaN
1,4f590f2ce4b0cd6f8d6828e8,Metro Madrid Línea 8 (Nuevos Ministerios - Aer...,{},"{'address': 'C. Gran Via de Hortaleza 49', 'la...","[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN
2,4bc2d53c2a89ef3bc242f488,Metro Aeropuerto T1-T2-T3,{},"{'address': 'Terminal 2', 'crossStreet': 'MAD ...","[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN
3,5b982c4cba57b4002c48bfe5,Transfer Aeropuerto Madrid - Transporte Coyoacan,{},"{'address': 'Calle LIBERTAD, 4', 'lat': 40.420...","[{'id': '54541b70498ea6ccd0204bff', 'name': 'T...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN
4,4e24dcb788772beda9dcbc40,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,{},"{'address': 'Paseo del Prado', 'lat': 40.41904...","[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN
5,4bc2d4ab2a89ef3b6742f488,Cercanías Aeropuerto T4,{},"{'address': 'Terminal 4', 'crossStreet': 'Aero...","[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN
6,5f574011a4af211b8cd1d400,Aeropuerto Madrid Barajas,{},{'address': 'Calle del Príncipe de Vergara 53'...,"[{'id': '53fca564498e1a175f32528b', 'name': 'T...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,{'id': '591746729'}
7,4bc70ab6af07a5930b647e2d,Aeropuerto de Madrid-Cuatro Vientos (MCV),{},{'address': 'Ctra de Acceso al Barrio de la Fo...,"[{'id': '4bf58dd8d48988d1ed931735', 'name': 'A...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN
8,5085849be88982ac6a09d6a4,Metro Aeropuerto T4,{},"{'address': 'Terminal 4', 'crossStreet': 'Aero...","[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN
9,4d1db7ed756e8cfaadcd5154,Parada O'Donnell - Línea Exprés Aeropuerto,{},"{'address': 'Calle de O'Donnell 61', 'lat': 40...","[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636887743,[],False,NaN


In [17]:
type(airports)

dict

In [18]:
airports["response"]["venues"][0]

{'id': '4afde737f964a520e72b22e3',
 'name': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
 'contact': {},
 'location': {'address': 'Av. de la Hispanidad, s/n',
  'lat': 40.4902221628413,
  'lng': -3.569434148638026,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.4902221628413,
    'lng': -3.569434148638026}],
  'distance': 11590,
  'postalCode': '28042',
  'cc': 'ES',
  'city': 'Madrid',
  'state': 'Madrid',
  'country': 'España',
  'formattedAddress': ['Av. de la Hispanidad, s/n',
   '28042 Madrid Madrid',
   'España']},
 'categories': [{'id': '4bf58dd8d48988d1ed931735',
   'name': 'Airport',
   'pluralName': 'Airports',
   'shortName': 'Airport',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
 

In [19]:
aeropuerto = airports["response"]["venues"]

In [20]:
extraer_air = cf.extraetodo(aeropuerto)

In [21]:
extraer_air

[{'nombre': 'Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) (Aeropuerto Adolfo Suárez Madrid-Barajas)',
  'latitud': 40.4902221628413,
  'longitud': -3.569434148638026,
  'location': {'type': 'Point',
   'coordinates': [40.4902221628413, -3.569434148638026]}},
 {'nombre': 'Metro Madrid Línea 8 (Nuevos Ministerios - Aeropuerto)',
  'latitud': 40.445477752377386,
  'longitud': -3.691766929219179,
  'location': {'type': 'Point',
   'coordinates': [40.445477752377386, -3.691766929219179]}},
 {'nombre': 'Metro Aeropuerto T1-T2-T3',
  'latitud': 40.4682787651721,
  'longitud': -3.5712555983647887,
  'location': {'type': 'Point',
   'coordinates': [40.4682787651721, -3.5712555983647887]}},
 {'nombre': 'Transfer Aeropuerto Madrid - Transporte Coyoacan',
  'latitud': 40.420315457,
  'longitud': -3.69735505,
  'location': {'type': 'Point', 'coordinates': [40.420315457, -3.69735505]}},
 {'nombre': 'Parada Plaza De Cibeles - Línea Exprés Aeropuerto',
  'latitud': 40.41904743901457,
  'longitud': -3

In [22]:
df_airp = pd.DataFrame(extraer_air)
df_airp.head()

,nombre,latitud,longitud,location
0,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
1,Metro Madrid Línea 8 (Nuevos Ministerios - Aer...,40.445478,-3.691767,"{'type': 'Point', 'coordinates': [40.445477752..."
2,Metro Aeropuerto T1-T2-T3,40.468279,-3.571256,"{'type': 'Point', 'coordinates': [40.468278765..."
3,Transfer Aeropuerto Madrid - Transporte Coyoacan,40.420315,-3.697355,"{'type': 'Point', 'coordinates': [40.420315457..."
4,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,40.419047,-3.692954,"{'type': 'Point', 'coordinates': [40.419047439..."


In [23]:
len(df_airp)

30

### Party

In [24]:
parametros_party = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "Discoteca"
}

In [25]:
disco = requests.get(url_query, params = parametros_party).json()

In [26]:
disco["response"].keys()

dict_keys(['venues'])

In [27]:
discoteca = disco["response"]["venues"]

In [28]:
extraer_disco = cf.extraetodo(discoteca)

In [29]:
df_disco = pd.DataFrame(extraer_disco)
df_disco.head()

,nombre,latitud,longitud,location
0,Discoteca Nells Madrid,40.439664,-3.682127,"{'type': 'Point', 'coordinates': [40.439664, -..."
1,Discoteca Palace,40.418451,-3.709102,"{'type': 'Point', 'coordinates': [40.418450979..."
2,Discoteca Almontes Sala Rociera,40.432799,-3.678725,"{'type': 'Point', 'coordinates': [40.43279897,..."
3,la cubana discoteca,40.450147,-3.695196,"{'type': 'Point', 'coordinates': [40.450147, -..."
4,Discoteca BAO-BAB Madrid,40.453919,-3.677663,"{'type': 'Point', 'coordinates': [40.453919368..."


### Vegan Restaurants

In [30]:
parametros_vegan = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "vegan"
}

In [31]:
vegan = requests.get(url_query, params = parametros_vegan).json()

In [32]:
vegano = vegan["response"]["venues"]

In [33]:
extraer_vegan = cf.extraetodo(vegano)

In [34]:
df_vegan = pd.DataFrame(extraer_vegan)
df_vegan.head()

,nombre,latitud,longitud,location
0,Vegan Streets Madrid Castellana,40.442456,-3.679973,"{'type': 'Point', 'coordinates': [40.4424564, ..."
1,Vegan Streets Madrid Centro,40.417512,-3.697636,"{'type': 'Point', 'coordinates': [40.4175117, ..."
2,vegetarian / vegan restaurant,40.432974,-3.686693,"{'type': 'Point', 'coordinates': [40.432974, -..."
3,Mad Mad Vegan,40.424183,-3.700731,"{'type': 'Point', 'coordinates': [40.424183, -..."
4,Los Andenes Vegan,40.435206,-3.710348,"{'type': 'Point', 'coordinates': [40.435206, -..."


### Nursery

In [35]:
parametros_nurs = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Madrid['coordinates'][0]}, {Madrid['coordinates'][1]}",
    "query": "guarderías"
}

In [36]:
nursery = requests.get(url_query, params = parametros_nurs).json()

In [37]:
guarderia = nursery["response"]["venues"]

In [38]:
extraer_nursery = cf.extraetodo(guarderia)

In [39]:
df_nursery = pd.DataFrame(extraer_nursery)
df_nursery.head()

,nombre,latitud,longitud,location
0,Guarderia San Nicolas,40.421913,-3.668392,"{'type': 'Point', 'coordinates': [40.421913, -..."
1,Guardería San Roque,40.389522,-3.736510,"{'type': 'Point', 'coordinates': [40.389521950..."
2,guarderia Santa Catalina,40.458316,-3.779061,"{'type': 'Point', 'coordinates': [40.458316239..."
3,Guarderia santa isabel,40.542532,-3.637349,"{'type': 'Point', 'coordinates': [40.542531541..."


### unión de las 4 listas de requerimientos para Madrid

In [40]:
MADRID = cf.suma_requisitos(extraer_air,extraer_disco,extraer_vegan,extraer_nursery)

In [41]:
df_MADRID = pd.DataFrame(MADRID)
df_MADRID

,nombre,latitud,longitud,location
0,Aeropuerto Adolfo Suárez Madrid-Barajas (MAD) ...,40.490222,-3.569434,"{'type': 'Point', 'coordinates': [40.490222162..."
1,Metro Madrid Línea 8 (Nuevos Ministerios - Aer...,40.445478,-3.691767,"{'type': 'Point', 'coordinates': [40.445477752..."
2,Metro Aeropuerto T1-T2-T3,40.468279,-3.571256,"{'type': 'Point', 'coordinates': [40.468278765..."
3,Transfer Aeropuerto Madrid - Transporte Coyoacan,40.420315,-3.697355,"{'type': 'Point', 'coordinates': [40.420315457..."
4,Parada Plaza De Cibeles - Línea Exprés Aeropuerto,40.419047,-3.692954,"{'type': 'Point', 'coordinates': [40.419047439..."
...,...,...,...,...
89,Inmbiliaria Vega New House,40.391291,-3.724951,"{'type': 'Point', 'coordinates': [40.391291, -..."
90,Guarderia San Nicolas,40.421913,-3.668392,"{'type': 'Point', 'coordinates': [40.421913, -..."
91,Guardería San Roque,40.389522,-3.736510,"{'type': 'Point', 'coordinates': [40.389521950..."
92,guarderia Santa Catalina,40.458316,-3.779061,"{'type': 'Point', 'coordinates': [40.458316239..."


In [42]:
with open ("MADRID.json","w") as f: #NOMBRO MI JSON VACÍO CREADO EN MONGO
    json.dump(MADRID,f) 

In [43]:
total_madrid = df_MADRID.copy()

In [44]:
client = MongoClient("localhost:27017")
db = client.get_database("PROYECTO3_GEOQ") #me conecto a la base de datos creada en Mongo con toda la info de Madrid
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO3_GEOQ')

In [45]:
conect_madrid = db.get_collection("MADRID") #me conecto a la colección de Madrid de la BBDD de Mongo, la re

In [46]:
index_madrid = db.MADRID.create_index([("location", GEOSPHERE)]) #creo un índice

In [47]:
Madrid #mis coordenadas iniciales para el punto elegido en Madrid

{'type': 'Point', 'coordinates': ['40.44242', '-3.69101']}

In [52]:
Madrid_alreves = cf.type_point ([-3.69101,40.44242])

In [53]:
Madrid_alreves

{'type': 'Point', 'coordinates': [-3.69101, 40.44242]}

In [62]:
Madrid_bien = cf.type_point ([40.44242,-3.69101])

In [132]:
metros = 5000

In [133]:
query = {"location": {"$near": {"$geometry": Madrid_bien, "$maxDistance": metros}}}

In [134]:
query

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [40.44242, -3.69101]},
   '$maxDistance': 5000}}}

In [135]:
list(conect_madrid.find(query))

[{'_id': ObjectId('618ffcb5096c866a87c3ba52'),
  'nombre': 'Metro Madrid Línea 8 (Nuevos Ministerios - Aeropuerto)',
  'latitud': 40.445477752377386,
  'longitud': -3.691766929219179,
  'location': {'type': 'Point',
   'coordinates': [40.445477752377386, -3.691766929219179]}},
 {'_id': ObjectId('618ffcb5096c866a87c3ba6b'),
  'nombre': 'Metro Nuevos Ministerios',
  'latitud': 40.44619324375705,
  'longitud': -3.6923790149464097,
  'location': {'type': 'Point',
   'coordinates': [40.44619324375705, -3.6923790149464097]}},
 {'_id': ObjectId('618ffcb5096c866a87c3ba8a'),
  'nombre': 'La Nuit Summertime',
  'latitud': 40.448908968322655,
  'longitud': -3.695329046143475,
  'location': {'type': 'Point',
   'coordinates': [40.448908968322655, -3.695329046143475]}},
 {'_id': ObjectId('618ffcb5096c866a87c3ba72'),
  'nombre': 'la cubana discoteca',
  'latitud': 40.450147,
  'longitud': -3.695196,
  'location': {'type': 'Point', 'coordinates': [40.450147, -3.695196]}},
 {'_id': ObjectId('618ffcb50

In [138]:
type(df_MADRID)

pandas.core.frame.DataFrame

In [139]:
df_MADRID.sample()

,nombre,latitud,longitud,location
5,Cercanías Aeropuerto T4,40.491486,-3.593721,"{'type': 'Point', 'coordinates': [40.491486343..."


In [140]:
gdf_MADRID = gpd.GeoDataFrame(df_MADRID, geometry=gpd.points_from_xy(df_MADRID.longitud, df_MADRID.latitud))

In [141]:
gdf_MADRID.sample()

,nombre,latitud,longitud,location,geometry
9,Parada O'Donnell - Línea Exprés Aeropuerto,40.421182,-3.669248,"{'type': 'Point', 'coordinates': [40.421181612...",POINT (-3.66925 40.42118)


In [142]:
type(gdf_MADRID)

geopandas.geodataframe.GeoDataFrame

In [145]:
Map(Layer(gdf_MADRID, "color:blue", popup_hover=[popup_element("nombre", "Propuestas Madrid")]))

## MALLORCA

In [79]:
Palma = cf.geocode("P.º de Mallorca, 7, 07011 Palma, Islas Baleares")

In [80]:
Palma

{'type': 'Point', 'coordinates': ['39.57355', '2.64177']}

In [82]:
Palma_bien = cf.type_point ([39.57355,2.64177])

### Airports

In [83]:
parametros_airp_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [84]:
airports_p = requests.get(url_query, params = parametros_airp_p).json()

In [85]:
airports_p["response"]["venues"][0]["name"]

'Aeropuerto de Mallorca'

In [86]:
aeropuerto_p = airports_p["response"]["venues"]

In [87]:
extraer_air_p = cf.extraetodo(aeropuerto_p)

### Party

In [88]:
parametros_party_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "Discoteca"
}

In [89]:
disco_p = requests.get(url_query, params = parametros_party_p).json()

In [90]:
discoteca_p = disco_p["response"]["venues"]

In [91]:
extraer_disco_p = cf.extraetodo(discoteca_p)

### Vegan Restaurants

In [92]:
parametros_vegan_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "vegan"
}

In [93]:
vegan_p = requests.get(url_query, params = parametros_vegan_p).json()

In [94]:
vegano_p = vegan_p["response"]["venues"]

In [95]:
extraer_vegan_p = cf.extraetodo(vegano_p)

### Nursery

In [96]:
parametros_nurs_p = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Palma['coordinates'][0]}, {Palma['coordinates'][1]}",
    "query": "guarderías"
}

In [97]:
nursery_p = requests.get(url_query, params = parametros_nurs_p).json()

In [98]:
guarderia_p = nursery_p["response"]["venues"]

In [99]:
extraer_nursery_p = cf.extraetodo(guarderia_p)

### unión de las 4 listas de requerimientos para Mallorca

In [100]:
MALLORCA = cf.suma_requisitos(extraer_air_p,extraer_disco_p,extraer_vegan_p,extraer_nursery_p)

In [101]:
df_MALLORCA = pd.DataFrame(MALLORCA)
df_MALLORCA.head()

,nombre,latitud,longitud,location
0,Aeropuerto de Mallorca,39.547252,2.730011,"{'type': 'Point', 'coordinates': [39.547252019..."
1,Sixt Alquiler de Coches | Palma De Mallorca Ae...,39.547846,2.731282,"{'type': 'Point', 'coordinates': [39.5478457, ..."
2,Aeropuerto Son Sant Joan puerta D 88,39.570960,2.670012,"{'type': 'Point', 'coordinates': [39.570959584..."
3,Cantina Aeropuerto,39.550482,2.730585,"{'type': 'Point', 'coordinates': [39.550482166..."
4,Aeroport de Palma de Mallorca (PMI) (Aeropuert...,39.548185,2.731743,"{'type': 'Point', 'coordinates': [39.548185019..."


In [102]:
with open ("MALLORCA.json","w") as f: #NOMBRO MI JSON VACÍO CREADO EN MONGO
    json.dump(MALLORCA,f) 

In [103]:
#client = MongoClient("localhost:27017")
#db2 = client.get_database("PROYECTO3_GEOQ") #me conecto a la base de datos creada en Mongo con toda la info de Madrid
#db2

In [104]:
conect_mallorca = db.get_collection("MALLORCA")

In [146]:
index_mallorca = db.MALLORCA.create_index([("location", GEOSPHERE)])

In [147]:
query2 = {"location": {"$near": {"$geometry": Palma_bien, "$maxDistance": metros}}}

In [148]:
query2

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [39.57355, 2.64177]},
   '$maxDistance': 5000}}}

In [149]:
list(conect_mallorca.find(query2))

[{'_id': ObjectId('6190fb91096c866a87c3bae0'),
  'nombre': 'LoVegano',
  'latitud': 39.57294406832292,
  'longitud': 2.6380256461803793,
  'location': {'type': 'Point',
   'coordinates': [39.57294406832292, 2.6380256461803793]}},
 {'_id': ObjectId('6190fb91096c866a87c3badd'),
  'nombre': 'Sabores de Sandra VEGAN Restaurant',
  'latitud': 39.5770042,
  'longitud': 2.6377468,
  'location': {'type': 'Point', 'coordinates': [39.5770042, 2.6377468]}},
 {'_id': ObjectId('6190fb91096c866a87c3bade'),
  'nombre': 'Vegan & Raw',
  'latitud': 39.57587445446343,
  'longitud': 2.6486116647720337,
  'location': {'type': 'Point',
   'coordinates': [39.57587445446343, 2.6486116647720337]}},
 {'_id': ObjectId('6190fb91096c866a87c3badf'),
  'nombre': 'Sabor Vegan Culinary Academy',
  'latitud': 39.57387362978236,
  'longitud': 2.653283807045242,
  'location': {'type': 'Point',
   'coordinates': [39.57387362978236, 2.653283807045242]}},
 {'_id': ObjectId('6190fb91096c866a87c3bae1'),
  'nombre': 'Lo vegan

In [160]:
df_MALLORCA.sample()

,nombre,latitud,longitud,location
27,Duana,39.547305,2.733128,"{'type': 'Point', 'coordinates': [39.547305409..."


In [162]:
gdf_MALLORCA = gpd.GeoDataFrame(df_MALLORCA, geometry=gpd.points_from_xy(df_MALLORCA.longitud, df_MALLORCA.latitud))

In [164]:
gdf_MALLORCA.sample()

,nombre,latitud,longitud,location,geometry
5,Avis Alquiler de Coches Aeropuerto de Mallorca,39.547083,2.729319,"{'type': 'Point', 'coordinates': [39.547083310...",POINT (2.72932 39.54708)


In [163]:
Map(Layer(gdf_MALLORCA, "color:green", popup_hover=[popup_element("nombre", "Propuestas Mallorca")]))

## SALAMANCA

In [167]:
Salamanca = cf.geocode("C. Toro, 22, 37002 Salamanca")

In [168]:
Salamanca

{'type': 'Point', 'coordinates': ['40.96636', '-5.66301']}

### Airports

In [169]:
parametros_airp_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "Aeropuerto"
}

In [170]:
airports_s = requests.get(url_query, params = parametros_airp_s).json()

In [171]:
airports_s["response"]["venues"][0]["name"]

'Aeropuerto de Salamanca-Matacán (SLM)'

In [172]:
aeropuerto_s = airports_s["response"]["venues"]
aeropuerto_s

[{'id': '4c08e8de340720a1bdcb8393',
  'name': 'Aeropuerto de Salamanca-Matacán (SLM)',
  'contact': {},
  'location': {'address': 'Ctra Salamanca-Ávila',
   'lat': 40.94017220282839,
   'lng': -5.502113981972948,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.94017220282839,
     'lng': -5.502113981972948}],
   'distance': 13837,
   'postalCode': '37893',
   'cc': 'ES',
   'city': 'Villagonzalo de Tormes',
   'state': 'Castilla y León',
   'country': 'España',
   'formattedAddress': ['Ctra Salamanca-Ávila',
    '37893 Villagonzalo de Tormes Castilla y León',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d1ed931735',
    'name': 'Airport',
    'pluralName': 'Airports',
    'shortName': 'Airport',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'las

In [173]:
extraer_air_s = cf.extraetodo(aeropuerto_s)

### Party

In [174]:
parametros_party_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "Pub"
}

In [175]:
disco_s = requests.get(url_query, params = parametros_party_s).json()

In [176]:
discoteca_s = disco_s["response"]["venues"]
discoteca_s

[{'id': '5093d3bee4b03f1e90517a1e',
  'name': "King's Pub",
  'contact': {},
  'location': {'lat': 40.967073,
   'lng': -5.663044,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.967073,
     'lng': -5.663044}],
   'distance': 79,
   'cc': 'ES',
   'country': 'España',
   'formattedAddress': ['España']},
  'categories': [{'id': '4bf58dd8d48988d11b941735',
    'name': 'Pub',
    'pluralName': 'Pubs',
    'shortName': 'Pub',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'marked': False,
   'unconfirmedCount': 0},
  'hereNow': {'count': 0, 'summary': 'Nobody here', 'groups': []},
  'referralId': 'v-1636892517',
  'venueChains': [],
  'hasPerk': False},
 {'id': '50143304e4b035d9f49614e1',
  'name': 'Tee Pub',
  'contact': {},
  'l

In [177]:
extraer_disco_s = cf.extraetodo(discoteca_s)

### Vegan Restaurants

In [178]:
parametros_vegan_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "vegan"
}

In [179]:
vegan_s = requests.get(url_query, params = parametros_vegan_s).json()

In [180]:
vegano_s = vegan_s["response"]["venues"]
vegano_s

[{'id': '572744e4cd10108364f3f94e',
  'name': '269 Gastro Vegan',
  'contact': {},
  'location': {'address': 'Crespo Rascón, 11',
   'lat': 40.967159,
   'lng': -5.665964,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.967159,
     'lng': -5.665964}],
   'distance': 263,
   'cc': 'ES',
   'city': 'Salamanca',
   'state': 'Castilla y León',
   'country': 'España',
   'formattedAddress': ['Crespo Rascón, 11',
    'Salamanca Castilla y León',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'marked': False,
   'unconfirmedCount':

In [181]:
extraer_vegan_s = cf.extraetodo(vegano_s)

### Nursery

In [182]:
parametros_nurs_s = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{Salamanca['coordinates'][0]}, {Salamanca['coordinates'][1]}",
    "query": "Guarderia"
}

In [183]:
nursery_s = requests.get(url_query, params = parametros_nurs_s).json()

In [184]:
guarderia_s = nursery_s["response"]["venues"]
guarderia_s

[{'id': '5127b89be0e2075351fec664',
  'name': 'Guardería British Bubbles',
  'contact': {},
  'location': {'address': 'c/ Isidro Segovia, 7',
   'lat': 40.970825866502224,
   'lng': -5.665715932846069,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.970825866502224,
     'lng': -5.665715932846069}],
   'distance': 546,
   'postalCode': '37007',
   'cc': 'ES',
   'city': 'Salamanca',
   'state': 'Castilla y León',
   'country': 'España',
   'formattedAddress': ['c/ Isidro Segovia, 7',
    '37007 Salamanca Castilla y León',
    'España']},
  'categories': [{'id': '4f4532974b9074f6e4fb0104',
    'name': 'Daycare',
    'pluralName': 'Daycares',
    'shortName': 'Daycare',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/daycare_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'marked': Fals

In [185]:
extraer_nursery_s = cf.extraetodo(guarderia_s)

### unión de las 4 listas de requerimientos para Salamanca

In [186]:
SALAMANCA = cf.suma_requisitos(extraer_air_s,extraer_disco_s,extraer_vegan_s,extraer_nursery_s)

In [187]:
df_SALAMANCA = pd.DataFrame(SALAMANCA)
df_SALAMANCA.head()

,nombre,latitud,longitud,location
0,Aeropuerto de Salamanca-Matacán (SLM),40.940172,-5.502114,"{'type': 'Point', 'coordinates': [40.940172202..."
1,King's Pub,40.967073,-5.663044,"{'type': 'Point', 'coordinates': [40.967073, -..."
2,Tee Pub,40.967113,-5.662745,"{'type': 'Point', 'coordinates': [40.967113010..."
3,O'haras's irish pub,40.966010,-5.663938,"{'type': 'Point', 'coordinates': [40.96601, -5..."
4,Pub Monumental,40.968882,-5.663795,"{'type': 'Point', 'coordinates': [40.968881660..."


In [188]:
with open ("SALAMANCA.json","w") as f: #NOMBRO MI JSON VACÍO CREADO EN MONGO
    json.dump(SALAMANCA,f) 

In [189]:
conect_salamanca = db.get_collection("SALAMANCA")

In [190]:
index_salamanca = db.SALAMANCA.create_index([("location", GEOSPHERE)])

In [191]:
Salamanca

{'type': 'Point', 'coordinates': ['40.96636', '-5.66301']}

In [192]:
Salamanca_bien = cf.type_point ([40.96636,-5.66301])

In [193]:
query3 = {"location": {"$near": {"$geometry": Salamanca_bien, "$maxDistance": metros}}}

In [194]:
query3

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [40.96636, -5.66301]},
   '$maxDistance': 5000}}}

In [195]:
list(conect_salamanca.find(query3))

[{'_id': ObjectId('6190fb9b096c866a87c3baec'),
  'nombre': "King's Pub",
  'latitud': 40.967073,
  'longitud': -5.663044,
  'location': {'type': 'Point', 'coordinates': [40.967073, -5.663044]}},
 {'_id': ObjectId('6190fb9b096c866a87c3baeb'),
  'nombre': 'Tee Pub',
  'latitud': 40.96711301075625,
  'longitud': -5.66274462835484,
  'location': {'type': 'Point',
   'coordinates': [40.96711301075625, -5.66274462835484]}},
 {'_id': ObjectId('6190fb9b096c866a87c3baed'),
  'nombre': "O'haras's irish pub",
  'latitud': 40.96601,
  'longitud': -5.663938,
  'location': {'type': 'Point', 'coordinates': [40.96601, -5.663938]}},
 {'_id': ObjectId('6190fb9b096c866a87c3baee'),
  'nombre': 'Oharas irish pub',
  'latitud': 40.96585589708964,
  'longitud': -5.664116006614261,
  'location': {'type': 'Point',
   'coordinates': [40.96585589708964, -5.664116006614261]}},
 {'_id': ObjectId('6190fb9b096c866a87c3baf0'),
  'nombre': 'The Liverpool Pub',
  'latitud': 40.96402364388889,
  'longitud': -5.662056383

In [196]:
df_SALAMANCA.sample()

,nombre,latitud,longitud,location
9,Peter's Pub,40.96292,-5.661974,"{'type': 'Point', 'coordinates': [40.962919854..."


In [197]:
gdf_SALAMANCA = gpd.GeoDataFrame(df_SALAMANCA, geometry=gpd.points_from_xy(df_SALAMANCA.longitud, df_SALAMANCA.latitud))

In [198]:
gdf_SALAMANCA.sample()

,nombre,latitud,longitud,location,geometry
34,Guardería Cari,40.952195,-5.627497,"{'type': 'Point', 'coordinates': [40.952194782...",POINT (-5.62750 40.95219)


In [199]:
Map(Layer(gdf_SALAMANCA, "color:red", popup_hover=[popup_element("nombre", "Propuestas Salamanca")]))

### Exporto en MongoDB

In [ ]:
#with open('aeropuerto_madrid.json', 'w') as f: #creo un json vacío donde voy a escribir (w) y donde voy a cargar mi lista de diccionarios "extraer_air"
    #json.dump(airports, f)

In [ ]:
pyjsonviewer.view_data(json_data=resp)